In [1]:
# This notebook is used to find a network from a list of nodes in Translator

In [1]:
import sys
sys.path.append('../TCT')
import TCT as TCT
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# Step1: List all the APIs in the translator system
Translator_KP_info,APInames= TCT.get_SmartAPI_Translator_KP_info()
#Translator_KP_info.to_csv('SmartAPI_Translator_KP_info.csv', index=False)
print(len(Translator_KP_info))

Skipping server without x-maturity: {'description': 'Production server', 'url': 'https://smart-api.info/api'}
Skipping server without x-maturity: {'description': 'Development server', 'url': 'https://dev.smart-api.info/api'}
SmartAPI API
Skipping server without production, staging or testing: {'description': 'Development server', 'url': 'https://dev.smart-api.info/api'}
Skipping server without x-maturity: {'description': 'Staging server', 'url': 'http://biothings.test.transltr.io/annotator'}
Skipping server without x-maturity: {'description': 'CI server', 'url': 'http://biothings.ci.transltr.io/annotator'}
99


In [3]:
# Step 2: Get metaKG and all predicates from Translator APIs
metaKG = TCT.get_KP_metadata(APInames) # This only applies to the Translator APIs
#print(metaKG.columns)
#print(metaKG.shape)

#metaKG.to_csv('../metaData/metaKG.csv', index=False)
#metaKG = pd.read_csv('../metaData/metaKG.csv')
All_predicates = list(set(metaKG['KG_category']))

In [4]:
# Step 3: set input parameters

# Node1 for query
#input_node1 = 'NPM1'
#input_node1_id = TCT.get_curie(input_node1)
#print(input_node1_id)
input_node1_list = [TCT.get_curie('NPM1'), TCT.get_curie('FLT3'), TCT.get_curie('NRAS'), TCT.get_curie('BCL2')]
input_node1_category = ['biolink:Gene'] # Node: this has to be in a format of biolink:xxx

#Node2 for query
input_node2_list = []
input_node2_category = ['biolink:Gene']


# Get all predicates for the input node1 and node2, user can furter select the predicates among this list
sele_predicates = list(set(TCT.select_concept(sub_list=input_node1_category,obj_list=input_node2_category,metaKG=metaKG)))
sele_predicates = ['biolink:genetically_associated_with', 'biolink:genetic_association', 
                   'biolink:genetically_interacts_with', 'biolink:physically_interacts_with',
                   'biolink:interacts_with',  'biolink:directly_physically_interacts_with','biolink:binds', ]
print("all relevant predicates in Translator:")
print(sele_predicates)
# select predicates

# Get all APIs for the input node1 and node2, user can furter select the APIs among this list
sele_APIs = TCT.select_API(sub_list=input_node1_category,obj_list=input_node2_category,metaKG=metaKG)
print("all relevant APIs in Translator:")
print(sele_APIs)
print(len(sele_APIs))

#sele_APIs = ['Multiomics BigGIM-DrugResponse KP API']
sele_APIs = sele_APIs
# get API URLs : two options: filter by API names or filter by predicates
API_URLs = TCT.get_Translator_API_URL(sele_APIs, 
                                      APInames)

all relevant predicates in Translator:
['biolink:genetically_associated_with', 'biolink:genetic_association', 'biolink:genetically_interacts_with', 'biolink:physically_interacts_with', 'biolink:interacts_with', 'biolink:directly_physically_interacts_with', 'biolink:binds']
all relevant APIs in Translator:
['MyGene.info API', 'Translator Annotation Service', 'Service Provider TRAPI', 'Automat-cam-kp(Trapi v1.5.0)', 'Automat-hetionet(Trapi v1.5.0)', 'Automat-ehr-may-treat-kp(Trapi v1.5.0)', 'PharmGKB REST API', 'SPOKE KP for TRAPI 1.5', 'Automat-icees-kg(Trapi v1.5.0)', 'Automat-robokop(Trapi v1.5.0)', 'MolePro', 'Automat-genome-alliance(Trapi v1.5.0)', 'imProving Agent for TRAPI 1.5', 'Automat-monarchinitiative(Trapi v1.5.0)', 'BioThings SEMMEDDB API', 'BioThings Explorer (BTE) TRAPI', 'Multiomics BigGIM-DrugResponse KP API', 'BioThings InnateDB API', 'Monarch API', 'Text Mining Targeted Association API', 'Automat-pharos(Trapi v1.5.0)', 'Connections Hypothesis Provider API']
22


In [5]:
len(API_URLs)

22

In [7]:
#URLS_sele = filter_APIs(sele_predicates, metaKG)

In [6]:
# Step 4: Format query json
query_json = TCT.format_query_json(input_node1_list,  # a list of identifiers for input node1
                                   input_node2_list,  # it can be empty list if only want to query node1
                                   input_node1_category,  # a list of categories of input node1
                                   input_node2_category,  # a list of categories of input node2
                                   sele_predicates) # a list of predicates


In [7]:
# Step 5: Query Translator APIs and parse results
result = TCT.parallel_api_query(API_URLs,query_json=query_json, max_workers=len(API_URLs))


Warning Code:422:https://automat.renci.org/ehr-may-treat-kp/query/
Warning Code:404:https://molepro-trapi.transltr.io/molepro/trapi/v1.5/query/
Warning Code:404:https://ia.transltr.io/api/v1.5/query/
Warning Code:422:https://automat.transltr.io/icees-kg/query/
Warning Code:404:https://api-v3.monarchinitiative.org/query/
Success!https://automat.transltr.io/monarch-kg/query/
Success!https://chp-api.transltr.io/query/
Warning Code:422:https://automat.transltr.io/genome-alliance/query/
Success!https://automat.transltr.io/cam-kp/query/
Success!https://automat.transltr.io/pharos/query/
Warning Code:404:https://spokekp.transltr.io/api/v1.5/query/
Warning Code:404:https://api.pharmgkb.org/v1/query/
Warning Code:400:https://biothings.ncats.io/annotator/query/Warning Code:400:https://biothings.ncats.io/innatedb/query/
Warning Code:400:https://biothings.ncats.io/biggim_drugresponse_kp/query/
Warning Code:400:https://biothings.ncats.io/semmeddb/query/
Warning Code:400:https://biothings.ncats.io/te

In [11]:
#result

In [8]:
result_df = TCT.parse_network_result(result, input_node1_list)

In [10]:
result_df

,Subject,Object
0,NCBIGene:4869,NCBIGene:4676
1,NCBIGene:4869,NCBIGene:54606
2,NCBIGene:4869,NCBIGene:2771
3,NCBIGene:4869,NCBIGene:142
4,NCBIGene:4869,NCBIGene:23411
...,...,...
3153,NCBIGene:909,NCBIGene:2322
3154,NCBIGene:6925,NCBIGene:596
3155,NCBIGene:6925,NCBIGene:2322
3156,NCBIGene:3091,NCBIGene:596


In [ ]:
TCT.visulize_path(TCT.get_curie("NPM1"), "NCBIGene:3320", TCT.get_curie("FLT3"), result, result)


CytoscapeWidget(cytoscape_layout={'name': 'cola', 'title': 'Path', 'nodeSpacing': 80, 'edgeLengthVal': 50}, cy…

,Subject,Object,Predicates,Subject_name,Object_name
0,NCBIGene:4869,NCBIGene:3320,genetically_interacts_with::infores:hetionet,NPM1,HSP90AA1
1,NCBIGene:4869,NCBIGene:3320,physically_interacts_with::infores:biogrid,NPM1,HSP90AA1
2,NCBIGene:4869,NCBIGene:3320,regulates::infores:text-mining-provider-targeted,NPM1,HSP90AA1
3,NCBIGene:4869,NCBIGene:3320,interacts_with::infores:biogrid,NPM1,HSP90AA1
4,NCBIGene:2322,NCBIGene:3320,interacts_with::infores:biogrid,FLT3,HSP90AA1
5,NCBIGene:2322,NCBIGene:3320,physically_interacts_with::infores:biogrid,FLT3,HSP90AA1


In [13]:
# results can be visualized in cytoscpe or networkx

In [14]:
#TCT.plot_graph_by_predicates(result_df)